## This Program gathers information for various sports that will be used to find the effect fans in thhe stands have on winning

#### Global Imports

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

# Basketball

## Imports

In [7]:
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc
from basketball_reference_scraper.seasons import get_schedule, get_standings

## Gathering Data

In [83]:
# Dicts that will be used to go between team names and acronyms. Useful for the API being used

team_to_acronym = {'Atlanta Hawks': 'ATL',
 'Boston Celtics': 'BOS',
 'Brooklyn Nets': 'NJN',
 'New Jersey Nets': 'NJN',
 'Chicago Bulls': 'CHI',
 'Charlotte Hornets (2014-present)': 'CHO',
 'Charlotte Bobcats': 'CHA',
 'Cleveland Cavaliers': 'CLE',
 'Dallas Mavericks': 'DAL',
 'Denver Nuggets': 'DEN',
 'Detroit Pistons': 'DET',
 'Golden State Warriors': 'GSW',
 'Houston Rockets': 'HOU',
 'Indiana Pacers': 'IND',
 'Los Angeles Clippers': 'LAC',
 'Los Angeles Lakers': 'LAL',
 'Memphis Grizzlies': 'MEM',
 'Miami Heat': 'MIA',
 'Milwaukee Bucks': 'MIL',
 'Minnesota Timberwolves': 'MIN',
 'New Orleans Pelicans': 'NOP',
 'New Orleans Hornets': 'NOH',
 'New York Knicks': 'NYK',
 'Oklahoma City Thunder': 'OKC',
 'Orlando Magic': 'ORL',
 'Philadelphia 76ers': 'PHI',
 'Phoenix Suns': 'PHO',
 'Portland Trail Blazers': 'POR',
 'Sacramento Kings': 'SAC',
 'San Antonio Spurs': 'SAS',
 'Toronto Raptors': 'TOR',
 'Utah Jazz': 'UTA',
 'Washington Wizards': 'WAS'}

acronym_to_team = {'ATL': 'Atlanta Hawks',
 'BOS': 'Boston Celtics',
 'BRK': 'Brooklyn Nets',
 'NJN': 'New jersey Nets',
 'CHI': 'Chicago Bulls',
 'CHO': 'Charlotte Hornets (2014-present)',
 'CHA': 'Charlotte Bobcats',
 'CLE': 'Cleveland Cavaliers',
 'DAL': 'Dallas Mavericks',
 'DEN': 'Denver Nuggets',
 'DET': 'Detroit Pistons',
 'GSW': 'Golden State Warriors',
 'HOU': 'Houston Rockets',
 'IND': 'Indiana Pacers',
 'LAC': 'Los Angeles Clippers',
 'LAL': 'Los Angeles Lakers',
 'MEM': 'Memphis Grizzlies',
 'MIA': 'Miami Heat',
 'MIL': 'Milwaukee Bucks',
 'MIN': 'Minnesota Timberwolves',
 'NOP': 'New Orleans Pelicans',
 'NOJ': 'New Orleans Pelicans',
 'NYK': 'New York Knicks',
 'OKC': 'Oklahoma City Thunder',
 'ORL': 'Orlando Magic',
 'PHI': 'Philadelphia 76ers',
 'PHO': 'Phoenix Suns',
 'POR': 'Portland Trail Blazers',
 'SAC': 'Sacramento Kings',
 'SAS': 'San Antonio Spurs',
 'TOR': 'Toronto Raptors',
 'UTA': 'Utah Jazz',
 'WAS': 'Washington Wizards'}

In [101]:
def basketball_data_generator():
    """ Uses API that connect to SportsReference.com to gather information related to NBA (basketball) team attendance. Note that this API
        only has average attendance so generalizations were made from an entire season to a single game
    
    Output: basketball_games_df: DataFrame of NBA game by game matchups, scores, and attendance
    """

    basketball_games_df = pd.DataFrame()   # DataFrame that will hold data
    start_year = 2010
    end_year = 2021

    for year in range(start_year, end_year+1):   # Iterating through desired years
        
        # Adding generic information
        game_data['Sport'] = 'Basketball'
        game_data['League'] = 'NBA'
        game_data['Season'] = year
        
        # Accessing API through basketball_reference_scraper library
        game_data = get_schedule(year)

        # Adding game-by-game information
        game_data.rename(columns={'DATE': 'Date', 'HOME': 'Home_Team', 'VISITOR': 'Away_Team',   # Renaming columns to names used in final DataFrame
                                  'HOME_PTS': 'Home_Team_Score', 'VISITOR_PTS': 'Away_Team_Score'},inplace=True)
        game_data['Winner'] = game_data.apply(lambda x: 'Home' if x['Home_Team_Score'] > x['Away_Team_Score'] 
                                                               else 'Away' if x['Home_Team_Score'] < x['Away_Team_Score'] else 'Tie', axis=1)
        game_data['Home_Team_Won'] = game_data.apply(lambda x: 1 if x['Winner'] == 'Home' else 0, axis=1)

        for team in acronym_to_team.keys():    # Going through all teams in the league to gather attendance information
            try:
                attendance_pg = get_team_misc(team, year)['ATTENDANCE/G']
                game_data.loc[game_data['Home_Team']==acronym_to_team[team], 'Attendance'] = attendance_pg
            except:
                continue

        game_data = game_data[['Sport', 'League', 'Date', 'Season', 'Home_Team', 'Away_Team', 'Home_Team_Score', 'Away_Team_Score', 'Winner', 'Home_Team_Won', 'Attendance']]
        basketball_games_df = basketball_games_df.append(game_data)
        basketball_games_df.drop_duplicates(inplace=True)

    # Accounting for nulls in attendance (no fans) and teams changing names over the years
    basketball_games_df['Attendance'].replace(np.nan, 0, regex=True, inplace=True)
    basketball_games_df['Home_Team'].replace('New Jersey Nets', 'Brooklyn Nets', regex=True, inplace=True)
    basketball_games_df['Away_Team'].replace('New Jersey Nets', 'Brooklyn Nets', regex=True, inplace=True)
    basketball_games_df['Home_Team'].replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True, inplace=True)
    basketball_games_df['Away_Team'].replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True, inplace=True)
    basketball_games_df['Home_Team'].replace('Charlotte Hornets (2014-present)', 'Charlotte Hornets', regex=True, inplace=True)
    basketball_games_df['Away_Team'].replace('Charlotte Hornets (2014-present)', 'Charlotte Hornets', regex=True, inplace=True)
    basketball_games_df['Home_Team'].replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True, inplace=True)
    basketball_games_df['Away_Team'].replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True, inplace=True)
    basketball_games_df.dropna(inplace=True)

    return basketball_games_df

ATL
16546
BOS
18624
BRK
BRK
NJN
13103
CHI
20726
CHO
CHO
CHA
15818
CLE
20562
DAL
19994
DEN
17983
DET
18752
GSW
18027
HOU
16528
IND
14202
LAC
16343
LAL
18997
MEM
13486
MIA
17730
MIL
15109
MIN
15102
NOP
NOP
NOJ
NOJ
NYK
19501
OKC
18004
ORL
17461
PHI
14225
PHO
17648
POR
20497
SAC
13254
SAS
18090
TOR
17897
UTA
19378
WAS
16205
ATL
15649
BOS
18624
BRK
BRK
NJN
13949
CHI
21792
CHO
CHO
CHA
15846
CLE
20112
DAL
20102
DEN
16989
DET
16660
GSW
18693
HOU
16191
IND
13538
LAC
17743
LAL
18997
MEM
14651
MIA
19779
MIL
15412
MIN
15243
NOP
NOP
NOJ
NOJ
NYK
19729
OKC
18148
ORL
18972
PHI
14752
PHO
17567
POR
20510
SAC
13890
SAS
18314
TOR
16592
UTA
19512
WAS
16792
ATL
15200
BOS
18624
BRK
BRK
NJN
13961
CHI
22161
CHO
CHO
CHA
14757
CLE
15927
DAL
20335
DEN
17029
DET
14413
GSW
18858
HOU
15363
IND
14169
LAC
19219
LAL
18997
MEM
15710
MIA
19935
MIL
14719
MIN
17491
NOP
NOP
NOJ
NOJ
NYK
19763
OKC
18203
ORL
18897
PHI
17503
PHO
15598
POR
20496
SAC
14508
SAS
18397
TOR
16836
UTA
19307
WAS
16728
ATL
15126
BOS
18624
BRK
17188
NJN


,Sport,Date,Home_Team,Away_Team,Home_Team_Score,Away_Team_Score,Winner,Home_Team_Won,Attendance
0,Basketball,2009-10-27,Cleveland Cavaliers,Boston Celtics,89,95,Away,0,20562.0
1,Basketball,2009-10-27,Dallas Mavericks,Washington Wizards,91,102,Away,0,19994.0
2,Basketball,2009-10-27,Portland Trail Blazers,Houston Rockets,96,87,Home,1,20497.0
3,Basketball,2009-10-27,Los Angeles Lakers,Los Angeles Clippers,99,92,Home,1,18997.0
4,Basketball,2009-10-28,Atlanta Hawks,Indiana Pacers,120,109,Home,1,16546.0
...,...,...,...,...,...,...,...,...,...
1075,Basketball,2021-05-16,Portland Trail Blazers,Denver Nuggets,NaN,NaN,Tie,0,0.0
1076,Basketball,2021-05-16,Sacramento Kings,Utah Jazz,NaN,NaN,Tie,0,0.0
1077,Basketball,2021-05-16,San Antonio Spurs,Phoenix Suns,NaN,NaN,Tie,0,1091.0
1078,Basketball,2021-05-16,Toronto Raptors,Indiana Pacers,NaN,NaN,Tie,0,811.0


## Writing Data to CSV

In [105]:
basketball_games_df.to_csv("C:\\Users\\Jack\\OneDrive\\Documents\\basketball_data.csv", index=False)

# Baseball

## Imports

In [ ]:
from baseball_scraper import baseball_reference

## Gathering Data

In [2]:
# Dicts that will be used to go between team names and acronyms. Useful for the API being used

acronym_to_city_dict = {'ATL': 'Atlanta',
                        'WSN': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'FLA': 'Miami',     # Changed name in 2012
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinnati',
                        'PIT': 'Pittsburgh',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SFG': 'San Francisco',
                        'COL': 'Colorado',
                        'SDP': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TBR': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KCR': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}
nickname_to_city_dict = {'braves': 'Atlanta',
                        'nationals': 'Washington',
                        'mets': 'New York Mets',
                        'phillies': 'Philadelphia',
                        'marlins': 'Miami',
                        'cardinals': 'St. Louis',
                        'brewers': 'Milwaukee',
                        'cubs': 'Chicago Cubs',
                        'reds': 'Cincinnati',
                        'pirates': 'Pittsburgh',
                        'dodgers': 'Los Angeles Dodgers',
                        'diamondbacks': 'Arizona',
                        'd-backs': 'Arizona',
                        'giants': 'San Francisco',
                        'rockies': 'Colorado',
                        'padres': 'San Diego',
                        'yankees': 'New York Yankees',
                        'rays': 'Tampa Bay',
                        'red sox': 'Boston',
                        'blue jays': 'Toronto',
                        'orioles': 'Baltimore',
                        'twins': 'Minnesota',
                        'indians': 'Cleveland',
                        'white sox': 'Chicago White Sox',
                        'royals': 'Kansas City',
                        'tigers': 'Detroit',
                        'astros': 'Houston',
                        'athletics': 'Oakland',
                        'rangers': 'Texas',
                        'angels': 'Los Angeles Angels',
                        'mariners': 'Seattle'}

In [90]:
def baseball_data_generator():
    """ Uses API that connects to SportsReference.com to gather information related to MLB (baseball) team attendance. 
    
    Output: baseball_games_df: DataFrame of MLB game by game matchups, scores, and attendance
    """


    s = baseball_reference.TeamScraper()  # calling the API's scraping tool
    baseball_games_df = pd.DataFrame()    # DataFrame that will hold data
    start_year = 2010
    end_year = 2021

    for year in range(start_year, end_year+1):  # Iterating through desired years

        # Adding generic information
        game_data['Sport'] = 'Baseball'
        game_data['League'] = 'MLB'
        game_data['Season'] = year


        # Accessing API through basketball_reference_scraper library
        s.set_season(year)  # Calling API for a given year  

        for team in acronym_to_city_dict.keys():  # Scraping data for all teams

            try:   
                game_data = s.scrape(team)
                game_data['Home_Team'] = game_data.apply(lambda x: acronym_to_city_dict[x['Opp']] if x['Home_Away'] == '@' else acronym_to_city_dict[x['Tm']], axis=1)
                game_data['Home_Team_Score'] = game_data.apply(lambda x: x['RA'] if x['Home_Away'] == '@' else x['R'], axis=1)
                game_data['Away_Team'] = game_data.apply(lambda x: acronym_to_city_dict[x['Opp']] if x['Home_Away'] == 'Home' else acronym_to_city_dict[x['Tm']], axis=1)
                game_data['Away_Team_Score'] = game_data.apply(lambda x: x['RA'] if x['Home_Away'] == 'Home' else x['R'], axis=1)
                game_data['Winner'] = game_data.apply(lambda x: 'Home' if x['Home_Team_Score'] > x['Away_Team_Score'] else 'Away' if x['Home_Team_Score'] < x['Away_Team_Score'] else 'Tie', axis=1)
                game_data['Home_Team_Won'] = game_data.apply(lambda x: 1 if x['Winner'] == 'Home' else 0, axis=1)
                game_data = game_data[['Sport', 'League', 'Date', 'Home_Team', 'Away_Team', 'Home_Team_Score', 'Away_Team_Score', 'Winner', 'Home_Team_Won', 'Attendance']]
                baseball_games_df = baseball_games_df.append(game_data)
                baseball_games_df.drop_duplicates(inplace=True)
            except:
                continue

    # Accounting for nulls in attendance (no fans) and teams changing names over the years
    baseball_games_df['Attendance'].replace(np.nan, 0, regex=True, inplace=True)        
    baseball_games_df['Home_Team'].replace('FLA', 'MIA', regex=True, inplace=True)
    baseball_games_df['Away_Team'].replace('FLA', 'MIA', regex=True, inplace=True)
    baseball_games_df.dropna(inplace=True)

    return baseball_games_df

## Writing Data to CSV

In [13]:
baseball_games_df.to_csv("C:\\Users\\Jack\\OneDrive\\Documents\\baseball_data.csv", index=False)

# Soccer

# American Football. Data is loaded here and Joined in SQL

## Loading in files from GITHUB with Data

In [92]:
attendance = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-04/attendance.csv')
standings = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-04/standings.csv')
games = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-04/games.csv')

games.head()

,year,week,home_team,away_team,winner,tie,day,date,time,pts_win,pts_loss,yds_win,turnovers_win,yds_loss,turnovers_loss,home_team_name,home_team_city,away_team_name,away_team_city
0,2000,1,Minnesota Vikings,Chicago Bears,Minnesota Vikings,NaN,Sun,September 3,1:00PM,30,27,374,1,425,1,Vikings,Minnesota,Bears,Chicago
1,2000,1,Kansas City Chiefs,Indianapolis Colts,Indianapolis Colts,NaN,Sun,September 3,1:00PM,27,14,386,2,280,1,Chiefs,Kansas City,Colts,Indianapolis
2,2000,1,Washington Redskins,Carolina Panthers,Washington Redskins,NaN,Sun,September 3,1:01PM,20,17,396,0,236,1,Redskins,Washington,Panthers,Carolina
3,2000,1,Atlanta Falcons,San Francisco 49ers,Atlanta Falcons,NaN,Sun,September 3,1:02PM,36,28,359,1,339,1,Falcons,Atlanta,49ers,San Francisco
4,2000,1,Pittsburgh Steelers,Baltimore Ravens,Baltimore Ravens,NaN,Sun,September 3,1:02PM,16,0,336,0,223,1,Steelers,Pittsburgh,Ravens,Baltimore


## Formatting dates to traditional mm/dd/yyy format

In [93]:
games['date'] = games.apply(lambda x: datetime.strptime(x['date'].split()[0]+'/'+str(x['date'].split()[1])+'/'+str(x['year']), '%B/%d/%Y'), axis=1)

0      2000-09-03
1      2000-09-03
2      2000-09-03
3      2000-09-03
4      2000-09-03
          ...    
5319   2019-01-12
5320   2019-01-12
5321   2019-01-19
5322   2019-01-19
5323   2019-02-02
Name: date, Length: 5324, dtype: datetime64[ns]

## Updating home teams in attendance data to match format from games DataFrame

In [45]:
attendance.rename(columns={"home": 'home_team'}, inplace=True)
attendance['home_team'] = attendance.apply(lambda x: x['team'] + ' ' + x['team_name'], axis=1)
attendance.head()

,team,team_name,year,total,home_team,away,week,weekly_attendance
0,Arizona,Cardinals,2000,893926,Arizona Cardinals,506451,1,77434.0
1,Arizona,Cardinals,2000,893926,Arizona Cardinals,506451,2,66009.0
2,Arizona,Cardinals,2000,893926,Arizona Cardinals,506451,3,NaN
3,Arizona,Cardinals,2000,893926,Arizona Cardinals,506451,4,71801.0
4,Arizona,Cardinals,2000,893926,Arizona Cardinals,506451,5,66985.0


## Writing Data to CSV

In [68]:
attendance.to_csv("C:\\Users\\Jack\\OneDrive\\Documents\\nfl_attendance.csv", index=False)
games.to_csv("C:\\Users\\Jack\\OneDrive\\Documents\\nfl_games.csv", index=False)

# Combining Sports

## Gathering CSV's

In [87]:
baseball = pd.read_csv("C:\\Users\\Jack\\OneDrive\\Documents\\baseball_data.csv")
basketball = pd.read_csv("C:\\Users\\Jack\\OneDrive\\Documents\\basketball_data.csv")
soccer = pd.read_csv("C:\\Users\\Jack\\OneDrive\\Documents\\soccer_data.csv")
football = pd.read_csv("C:\\Users\\Jack\\OneDrive\\Documents\\american_football_data.csv")
hockey = pd.read_csv("C:\\Users\\Jack\\OneDrive\\Documents\\hockey_data.csv")  # Hockey data pulled directly from SportsReference.com

## Removing NBA games that were played in a neutral court "bubble"

In [88]:
basketball['Date'] = pd.to_datetime(basketball['Date'])  
basketball = basketball.loc[(basketball['Date'] < '2020-4-1') | (basketball['Date'] > '2020-11-1')]
basketball['Date'] = basketball['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

basketball.shape

(13851, 11)

## Combining DataFrames

In [89]:
sports_df = baseball.append(basketball, ignore_index=True)
sports_df = sports_df.append(soccer, ignore_index=True)
sports_df = sports_df.append(football, ignore_index=True)
sports_df = sports_df.append(hockey, ignore_index=True)

sports_df.dropna(inplace=True)  # One hockey game reschedules but still in original hockey DataFrame

sports_df.head()

(63067, 11)
(63066, 11)


,Sport,Date,Season,Home_Team,Away_Team,Home_Team_Score,Away_Team_Score,Winner,Home_Team_Won,Attendance,During_COVID
0,Baseball,4/5/2010,2010,Atlanta,Chicago Cubs,16.0,5.0,Home,1,53081,0
1,Baseball,4/7/2010,2010,Atlanta,Chicago Cubs,3.0,2.0,Home,1,36170,0
2,Baseball,4/8/2010,2010,Atlanta,Chicago Cubs,0.0,2.0,Away,0,27443,0
3,Baseball,4/9/2010,2010,San Francisco,Atlanta,5.0,4.0,Home,1,42940,0
4,Baseball,4/10/2010,2010,San Francisco,Atlanta,2.0,7.0,Away,0,42985,0


## Writing Data to CSV

In [86]:
sports_df.to_csv("C:\\Users\\Jack\\OneDrive\\Documents\\sports_data.csv", index=False)